In [1]:
import cv2
import mediapipe as mp

In [4]:
video = cv2.VideoCapture(filename='videos/video1.mp4')
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter(filename='number_of_steps.mp4',
                         fourcc=cv2.VideoWriter_fourcc(*"DIVX"),
                         fps=25,
                         frameSize=(width, height))

mpPose = mp.solutions.pose
pose = mpPose.Pose()

drawing = mp.solutions.drawing_utils

adim_sayisi = 0
adim_atildi = False

while True:
    retval, img = video.read()
    if not retval: break
    
    
    results = pose.process(image=img).pose_landmarks
    
    landmarkList = []
    if results:
        
        
        drawing.draw_landmarks(image=img,
                               landmark_list=results,
                               connections=mpPose.POSE_CONNECTIONS)
        for id, landmark in enumerate(results.landmark):
            height, width, color = img.shape
            cx, cy = int(landmark.x * width), int(landmark.y * height)
            landmarkList.append([id, cx, cy])
        
      
        sag_ayak_y =  landmarkList[26][2]
        sol_ayak_y =  landmarkList[25][2]
        
        if sol_ayak_y > sag_ayak_y and not adim_atildi:
            adim_sayisi += 1
            adim_atildi = True
        elif sol_ayak_y < sag_ayak_y:
            adim_atildi = False
            
        cv2.putText(img=img,
                    text=f"Adim Sayisi: {adim_sayisi}",
                    org=(30, 30),
                    fontFace=cv2.FONT_HERSHEY_DUPLEX,
                    fontScale=0.7,
                    color=(0, 0, 255),
                    thickness=3)
    
    writer.write(image=img)
    #img = cv2.flip(src=img, flipCode=1)
    cv2.imshow(winname='Video', mat=img)
    if cv2.waitKey(30) == ord('q'): break

writer.release()
video.release()
cv2.destroyAllWindows()